In [1]:
!pip install -r requirements.txt

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from __future__ import print_function
import torch
from torchvision import datasets, transforms
import numpy as np
from WeakLearners import WongNeuralNetCIFAR10
import matplotlib.pyplot as plt


In [3]:
from Boosting import runBoosting

In [4]:
import utils

In [5]:
cuda = torch.device('cuda:0')

# Vanilla Wong Neural Net CIFAR10

In [6]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=100, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(
    testset, batch_size=300, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## Non Adv

In [6]:
cifar10_vanilla = WongNeuralNetCIFAR10()
cifar10_vanilla.fit(train_loader, test_loader, alpha = 0, epochs = 100, adv=False, epsilon=0)

NameError: name 'train_loader' is not defined

In [ ]:
cifar10_vanilla.plot_train(batchSize = 100)

In [ ]:
cifar10_vanilla.plot_val_accuracies(batchSize = 100)

In [ ]:
print(cifar10_vanilla)

## Adversarial

In [12]:
from AdversarialAttacks import attack_fgsm

In [13]:
maxSamples = int(2e6)

In [ ]:
cifar10_adv = WongNeuralNetCIFAR10(attack_eps=[0.01, 0.02, 0.05, 0.1])
cifar10_adv.fit(train_loader, test_loader, adv_train=True, val_attacks=[attack_fgsm], maxSample=maxSamples)

adv_train: True
Epoch 0
about to record accs [<function attack_fgsm at 0x7fb46c5d4048>]
about to attack <function attack_fgsm at 0x7fb46c5d4048>
losses, accs {'val': 2.1196863651275635, <function attack_fgsm at 0x7fb46c5d4048>: [2.158881425857544, 2.195096015930176, 2.288396120071411, 2.4038236141204834]} {'val': 0.24333333333333335, <function attack_fgsm at 0x7fb46c5d4048>: [0.20666666666666667, 0.16666666666666666, 0.09333333333333334, 0.016666666666666666]}
Val accuracy: 0.24333333333333335
Progress: 10000,  val accuracy: 0.2433


In [ ]:
cifar10_adv.accuracies

In [ ]:
path_head = 'results/plots/cifar10_wl/'
acc_file = path_head + f'wl_acc_maxSamples_{maxSamples}.png'
adv_acc_file = path_head + f'adv_acc_maxSamples_{maxSamples}.png'
loss_file = path_head + f'loss_maxSamples_{maxSamples}.png'
cifar10_adv.plot_accuracies(acc_file)
cifar10_adv.plot_loss(loss_file)
cifar10_adv.plot_adversarial_accuracies(adv_acc_file)